In [1]:
!pip install -q sentence-transformers langchain chromadb pypdf faiss-cpu \
langchain_community scikit-learn numpy mistralai langchain-mistralai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.4 MB/s eta

In [2]:
import os
from getpass import getpass


In [3]:
os.environ["MISTRAL_API_KEY"] = getpass("Enter your Mistral API Key: ")


Enter your Mistral API Key: ··········


In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/2022-GROUP-II_20SERVICES-NOTIFN_20FINAL2022122923474120230118102628 (1).pdf")  # upload PDF to Colab
documents = loader.load()

print(f"Total pages loaded: {len(documents)}")


Total pages loaded: 46


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)

chunks = text_splitter.split_documents(documents)
print(f"Total chunks created: {len(chunks)}")


Total chunks created: 300


In [7]:
from langchain_mistralai.embeddings import MistralAIEmbeddings

embedding_model = MistralAIEmbeddings(
    model="mistral-embed"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

In [9]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)

print("Vector store created successfully")


Vector store created successfully


In [10]:
query = "What is this document about?"

results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\n--- Chunk {i+1} ---")
    print(doc.page_content[:400])



--- Chunk 1 ---
2 
 
(The Details of Vacancies  i.e., Community-wise, PH-wise, EWS, Sports Category, Multi-
Zone / Zonal and Gender-wise (General/Women) may be seen at Annexure-I.)    
 
IMPORTANT NOTE: - The number of vacancies are subject to variation on intimation being 
received from the Department concerned.  Addition of vacancies if any, will be accepted only 
before the date of examination and an ad dendum

--- Chunk 2 ---
warranted by any unforeseen circumstances arising during the course of this process, or as 
deemed necessary by the Commission at any stage. 
 
 
 
 
HYDERABAD                 sd/-        
DATE: 29 /12 /2022                              SECRETARY

--- Chunk 3 ---
TOTAL 30 19 11 5 7 4 9 7 1 1 8 3 5 2 18 9 11 5 4 2 - 1 1 1 - 1 - 109 56 165 
 
 
 
 
BREAK UP OF VACANCIES FOR THE POST OF ASSISTANT SECTION OFFICER IN LEGISLATIVE SECRETARIAT 
 
POST CODE-11 
 
STATE 
OC EWS BC-A BC-B BC-C BC-D BC-E SC ST SPORTS PH TOTAL 
GRAND 
TOTAL 
G W G W G W G W G W G W G W G 

In [11]:
from langchain_mistralai.chat_models import ChatMistralAI


llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0
)


In [12]:
def generate_answer(query, retrieved_docs):
    context = "\n\n".join(
        [f"[Chunk {i+1}]: {doc.page_content}"
         for i, doc in enumerate(retrieved_docs)]
    )

    prompt = f"""
You are a factual assistant.

Answer the question using ONLY the context below.
If the answer is not present, say:
"Answer not found in the provided document."

CONTEXT:
{context}

QUESTION:
{query}

Include citations like [Chunk X].
"""

    response = llm.invoke(prompt)
    return response.content


In [53]:
SIMILARITY_THRESHOLD = 0.9

def retrieve_with_threshold(query, k=5):
    docs_with_scores = vectorstore.similarity_search_with_score(query, k=k)

    filtered_docs = []
    for doc, score in docs_with_scores:
        similarity = 1 / (1 + score)  # FAISS distance → similarity
        if similarity >= SIMILARITY_THRESHOLD:
            filtered_docs.append(doc)

    return filtered_docs


In [54]:
def rag_pipeline(query):
    retrieved_docs = retrieve_with_threshold(query)

    if len(retrieved_docs) == 0:
        return "Naku matram em telsu mastaaru !"

    return generate_answer(query, retrieved_docs)


In [55]:
query = "What is the qualification ?"
answer = rag_pipeline(query)
print(answer)


Naku matram em telsu mastaaru !
